In [1]:
from pygraylog.graylogapi import GraylogAPI
import json
import sys
import numpy as np
import pandas as pd
import datetime

In [2]:
#version = 0.0.1
#18.05.2018

log_level = 0

graylog_con={
        "user":"merger",
        "password":"pc@pM8rger",
        "url":"http://graylog-ui.roamability.com:9000/api",
        "offset":0,
        "limit":300
    }

api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

In [49]:
#Подготовка необходимых функций

def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()

#Список полей для формирования DF в xDR

fields=['timestamp',
    'm3ua_OPC','m3ua_DPC',
    'sccp_calling_digits','sccp_calling_ssn',
    'sccp_called_digits','sccp_called_ssn',
    'protocol_version','opCode_localValue','TCAP_type',
    'tcap_otid','tcap_dtid',
    'e164_msisdn','e212_imsi']

#Load data from GrayLog

def LoadGraylogResult(exportsearch_results):
        jsonResult = json.loads(exportsearch_results)
        outputResult = list()
        i=0
        if "total_results" in jsonResult:
            outLog("Graylog search found {} messages".format(jsonResult["total_results"]),0)

        # Get the results and store them
            for result in jsonResult["messages"]:  
                if "message" in  result and isinstance(result["message"], dict):
                    outputResult.append(result["message"])
                    i+=1
                    if i%100==0: outLog("loaded {} records from Graylog".format(i),0)
            outLog("Graylog search load {} messages".format(len(outputResult)),0)
        else:
            outLog("Graylog search error: {}".format(exportsearch_results),0)
        return outputResult
    
#Load data into df

def LoadGraylogResultToDataFrame(data):
    columns=list(data[0].keys())
    values = list(data[0].values())
    arr_len = len(values)
    df=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
    for i in range(len(data)-1):
        columns=list(data[i+1].keys())
        values = list(data[i+1].values())
        arr_len = len(values)
        df1=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
        df=pd.concat([df, df1],axis=0,join='outer')
    df=df.reset_index(drop=True)
    if 'tcap_otid' in df.columns:
        df['tcap_otid']=df['tcap_otid'].fillna(0)
    if 'tcap_dtid' in df.columns:
        df['tcap_dtid']=df['tcap_dtid'].fillna(0)
    return df

#Compile transactions

def xdr_compile(otid,ind,df_xdr,df):
    
    fields=['tcap_dtid','tcap_otid','TCAP_type']
    if all(item in df.columns for item in fields):
        dtid=111
        df_xdr_temp=df_xdr
        for i in range(ind-1,-1,-1):
            if df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['tcap_otid']==0\
                and (df.iloc[i]['TCAP_type']=='End' or df.iloc[i]['TCAP_type']=='Abort'):
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
                df_xdr_temp['ind']=i
                return df_xdr_temp
            elif df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['TCAP_type']=='Continue' and df.iloc[i]['tcap_otid']<>dtid:
                dtid=df.iloc[i]['tcap_otid']
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
            elif df.iloc[i]['TCAP_type']=='Continue'\
                and (df.iloc[i]['tcap_dtid']==otid or df.iloc[i]['tcap_otid']==otid or df.iloc[i]['tcap_dtid']==dtid or df.iloc[i]['tcap_otid']==dtid):
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
            elif (df.iloc[i]['TCAP_type']=='End' or df.iloc[i]['TCAP_type']=='Abort')\
                and (df.iloc[i]['tcap_dtid']==otid or df.iloc[i]['tcap_otid']==otid or df.iloc[i]['tcap_dtid']==dtid or df.iloc[i]['tcap_otid']==dtid):
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
                df_xdr_temp['ind']=i
                return df_xdr_temp
        df_xdr_temp['ind']=i
        return df_xdr_temp
    else:
        print 'Not all fields are in the DataFrame'

In [50]:
#Формирование запроса и выгрузка данных

#--------------------------------------------------------------------------------
#Form date and time
#--------------------------------------------------------------------------------
#format like 2017-03-06T16:28

#Manually set the date and time
dateFrom = datetime.datetime.strptime('2018-12-02T18:04', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-12-02T18:05', '%Y-%m-%dT%H:%M')

#The last period of time
#last_hours=6
#dateFrom = datetime.datetime.strptime((datetime.datetime.now()-datetime.timedelta(last_hours)).strftime("%Y-%m-%dT%H:%M"),'%Y-%m-%dT%H:%M')
#dateTo = datetime.datetime.strptime(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M"), '%Y-%m-%dT%H:%M')

#--------------------------------------------------------------------------------
#Form query
#--------------------------------------------------------------------------------
#Fields examples
#protocol:ss7
#tcap_otid:"a2:2a:0c:40"
#tcap_dtid:"a2:2a:0c:40"
#opCode_localValue:2
#TCAP_type:Begin
#e212_imsi:425019629870706
#tcap_result:1
#tcap_dialogue_service_user:2

#Wrong MTFSM MAP version
#query='protocol:ss7 AND tcap_result:1 AND tcap_dialogue_service_user:2 AND tcap_application_context_name:"0.4.0.0.1.0.25.2" AND (m3ua_OPC:1961 OR m3ua_OPC:1962)'

imsi='425019612541090'

query='protocol:ss7 AND e212_imsi:{} AND opCode_localValue:23 AND TCAP_type:Begin'.format(imsi)
query='protocol:ss7 AND e212_imsi:{} AND protocol_name:camel AND TCAP_type:Begin'.format(imsi) 

#--------------------------------------------------------------------------------
#Run the query
#--------------------------------------------------------------------------------

print query + '\n'
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo, offset=graylog_con["offset"], limit=graylog_con["limit"])
data=LoadGraylogResult(gres)

protocol:ss7 AND e212_imsi:425019612541090 AND protocol_name:camel AND TCAP_type:Begin

Graylog search found 2 messages
Graylog search load 2 messages


In [51]:
#Формирование нового запроса со списком TID и повторная выгрузка данных

tid_list=[]
str_temp=''

for i in range(len(data)):
    try:
        tid_list.append(data[i]['tcap_otid'])
    except:
        pass
    try:
        tid_list.append(data[i]['tcap_dtid'])
    except:
        pass
tid_list
tid_list=tuple(set(tid_list))

i=1
for a in tid_list:
    str_temp=str_temp + 'tcap_otid:' + '"' + a  + '"' + ' OR tcap_dtid:' + '"' + a  + '"'
    if i<len(tid_list):
        str_temp=str_temp + ' OR '
    i=i+1
query=str('protocol:ss7 AND (' + str_temp + ')')

print 'The Query:\n' + query +'\n'

gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo, offset=graylog_con["offset"], limit=graylog_con["limit"])
data=LoadGraylogResult(gres)

try:
    df=LoadGraylogResultToDataFrame(data)
except:
    outLog('Nothing to combine.',0)

The Query:
protocol:ss7 AND (tcap_otid:"b1:1b:04:fe" OR tcap_dtid:"b1:1b:04:fe" OR tcap_otid:"51:80:90:9e" OR tcap_dtid:"51:80:90:9e")

Graylog search found 7 messages
Graylog search load 7 messages


In [52]:
#Сборка датафрейма XDR, вывод на экран или выгрузка в csv елси нужно
#Ver.1 Выводит все найденные записи (опционально)

fields_for_xdr=['tcap_dtid','tcap_otid','TCAP_type']

if all(item in df.columns for item in fields_for_xdr):
    columns=list(df.keys())
    df_xdr_res=pd.DataFrame(columns=columns)
    for i in range(len(df)-1,-1,-1):
        if df.iloc[i]['TCAP_type']=='Begin':
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr_res=pd.concat([df_xdr_res,xdr_compile(df.iloc[i]['tcap_otid'],i,df_temp,df)],axis=0,join='outer')
else:
    print 'Not all fields are in the DataFrame'

df_xdr_res=df_xdr_res.reset_index(drop=True)

xdr_fields=[]
for a in fields:
    if a in df.columns:
        #xdr_fields+=a
        xdr_fields.append(a)

df_xdr_res[xdr_fields]
#df_xdr_res[xdr_fields].to_csv('xDR_result.csv',index=False)

,timestamp,m3ua_OPC,m3ua_DPC,sccp_calling_digits,sccp_calling_ssn,sccp_called_digits,sccp_called_ssn,protocol_version,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
0,2018-12-02T17:04:43.653Z,1961,2505,41789310202,146,97254120636,146,1,Begin,51:80:90:9e,0,41789310202,425019612541090
1,2018-12-02T17:04:43.893Z,2505,1961,97254120636,146,41789310202,146,1,Continue,a1:1a:0d:cf,51:80:90:9e,972545209730,NaN
2,2018-12-02T17:04:44.804Z,1961,2505,41789310202,146,97254120636,146,NaN,End,51:80:90:9e,a1:1a:0d:cf,NaN,NaN
3,2018-12-02T17:04:43.668Z,800,781,41789310202,146,97254120636,146,1,Begin,b1:1b:04:fe,0,41789310202,425019612541090
4,2018-12-02T17:04:43.843Z,782,800,97254120636,146,41789310202,146,1,Continue,00:00:f8:e8,b1:1b:04:fe,972545209730,NaN
5,2018-12-02T17:04:44.805Z,800,781,41789310202,146,97254120636,146,1,Continue,b1:1b:04:fe,00:00:f8:e8,NaN,NaN
6,2018-12-02T17:04:44.828Z,782,800,97254120636,146,41789310202,146,NaN,Abort,0,b1:1b:04:fe,NaN,NaN


In [53]:
#Сборка датафрейма XDR, вывод на экран или выгрузка в csv если нужно
#ver.2 Выводит только записи с заданным IMSI

fields_for_xdr=['tcap_dtid','tcap_otid','TCAP_type']

if all(item in df.columns for item in fields_for_xdr):
    columns=list(df.keys())
    df_xdr_res=pd.DataFrame(columnshttp://localhost:8889/notebooks/Documents/GITLAB/RB_BD/RB_xDR_Browser_Graylog_Python2.ipynb#=columns)
    for i in range(len(df)-1,-1,-1):
        if df.iloc[i]['TCAP_type']=='Begin':
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr_res=pd.concat([df_xdr_res,xdr_compile(df.iloc[i]['tcap_otid'],i,df_temp,df)],axis=0,join='outer')
else:
    print 'Not all fields are in the DataFrame'

xdr_fields=[]
for a in fields:
    if a in df.columns:
        #xdr_fields+=a
        xdr_fields.append(a)

ind=df_xdr_res.ind[df_xdr_res.e212_imsi==imsi]

df_xdr=pd.DataFrame()

for i in ind:
    df_temp=df_xdr_res[df_xdr_res.ind==i]
    df_xdr=pd.concat([df_xdr,df_temp],axis=0,join='outer')
    
df_xdr=df_xdr.set_index(['ind'])
df_xdr[xdr_fields]

,timestamp,m3ua_OPC,m3ua_DPC,sccp_calling_digits,sccp_calling_ssn,sccp_called_digits,sccp_called_ssn,protocol_version,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
ind,,,,,,,,,,,,,
2.0,2018-12-02T17:04:43.653Z,1961,2505,41789310202,146,97254120636,146,1,Begin,51:80:90:9e,0,41789310202,425019612541090
2.0,2018-12-02T17:04:43.893Z,2505,1961,97254120636,146,41789310202,146,1,Continue,a1:1a:0d:cf,51:80:90:9e,972545209730,NaN
2.0,2018-12-02T17:04:44.804Z,1961,2505,41789310202,146,97254120636,146,NaN,End,51:80:90:9e,a1:1a:0d:cf,NaN,NaN
0.0,2018-12-02T17:04:43.668Z,800,781,41789310202,146,97254120636,146,1,Begin,b1:1b:04:fe,0,41789310202,425019612541090
0.0,2018-12-02T17:04:43.843Z,782,800,97254120636,146,41789310202,146,1,Continue,00:00:f8:e8,b1:1b:04:fe,972545209730,NaN
0.0,2018-12-02T17:04:44.805Z,800,781,41789310202,146,97254120636,146,1,Continue,b1:1b:04:fe,00:00:f8:e8,NaN,NaN
0.0,2018-12-02T17:04:44.828Z,782,800,97254120636,146,41789310202,146,NaN,Abort,0,b1:1b:04:fe,NaN,NaN


In [55]:
xdr_fields_1 = list(xdr_fields)
xdr_fields_1.append('file')
df_xdr[xdr_fields_1]

,timestamp,m3ua_OPC,m3ua_DPC,sccp_calling_digits,sccp_calling_ssn,sccp_called_digits,sccp_called_ssn,protocol_version,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi,file
ind,,,,,,,,,,,,,,
2.0,2018-12-02T17:04:43.653Z,1961,2505,41789310202,146,97254120636,146,1,Begin,51:80:90:9e,0,41789310202,425019612541090,/mnt/tracing1ch/sigtran-public/sigtran-public_...
2.0,2018-12-02T17:04:43.893Z,2505,1961,97254120636,146,41789310202,146,1,Continue,a1:1a:0d:cf,51:80:90:9e,972545209730,NaN,/mnt/tracing1ch/sigtran-public/sigtran-public_...
2.0,2018-12-02T17:04:44.804Z,1961,2505,41789310202,146,97254120636,146,NaN,End,51:80:90:9e,a1:1a:0d:cf,NaN,NaN,/mnt/tracing1ch/sigtran-public/sigtran-public_...
0.0,2018-12-02T17:04:43.668Z,800,781,41789310202,146,97254120636,146,1,Begin,b1:1b:04:fe,0,41789310202,425019612541090,/mnt/tracing1ch/sigtran-public/sigtran-public_...
0.0,2018-12-02T17:04:43.843Z,782,800,97254120636,146,41789310202,146,1,Continue,00:00:f8:e8,b1:1b:04:fe,972545209730,NaN,/mnt/tracing1ch/sigtran-public/sigtran-public_...
0.0,2018-12-02T17:04:44.805Z,800,781,41789310202,146,97254120636,146,1,Continue,b1:1b:04:fe,00:00:f8:e8,NaN,NaN,/mnt/tracing1ch/sigtran-public/sigtran-public_...
0.0,2018-12-02T17:04:44.828Z,782,800,97254120636,146,41789310202,146,NaN,Abort,0,b1:1b:04:fe,NaN,NaN,/mnt/tracing1ch/sigtran-public/sigtran-public_...


In [56]:
df_xdr[xdr_fields_1].to_csv('xDR_result.csv')